# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 10:56AM,241712,15,8265094,"Carwin Pharmaceutical Associates, LLC",Released
1,Jul 13 2022 10:56AM,241712,15,8265095,"Carwin Pharmaceutical Associates, LLC",Released
2,Jul 13 2022 10:56AM,241712,15,8265096,"Carwin Pharmaceutical Associates, LLC",Released
3,Jul 13 2022 10:56AM,241712,15,8265097,"Carwin Pharmaceutical Associates, LLC",Released
4,Jul 13 2022 10:56AM,241712,15,8265122,"Carwin Pharmaceutical Associates, LLC",Released
5,Jul 13 2022 10:56AM,241712,15,8265123,"Carwin Pharmaceutical Associates, LLC",Released
6,Jul 13 2022 10:55AM,241759,16,SSF-8269542,Sartorius Stedim Filters Inc.,Released
7,Jul 13 2022 10:53AM,241710,15,8264544,"Brookfield Pharmaceuticals, LLC",Released
8,Jul 13 2022 10:53AM,241710,15,8264545,"Brookfield Pharmaceuticals, LLC",Released
9,Jul 13 2022 10:53AM,241710,15,8264546,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,241755,Released,1
49,241756,Released,1
50,241757,Released,1
51,241758,Released,1
52,241759,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241755,NaN,NaN,1.0
241756,NaN,NaN,1.0
241757,NaN,NaN,1.0
241758,NaN,NaN,1.0
241759,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0.0,0.0,22.0
241648,1.0,0.0,4.0
241654,0.0,0.0,1.0
241657,0.0,0.0,1.0
241666,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0,0,22
241648,1,0,4
241654,0,0,1
241657,0,0,1
241666,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,0,0,22
1,241648,1,0,4
2,241654,0,0,1
3,241657,0,0,1
4,241666,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,,,22
1,241648,1,,4
2,241654,,,1
3,241657,,,1
4,241666,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC"
6,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.
7,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC"
33,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.
34,Jul 13 2022 10:46AM,241757,10,Beach Products Inc
35,Jul 13 2022 10:43AM,241756,19,"GCH Granules USA, Inc."
36,Jul 13 2022 10:36AM,241736,10,Bio-PRF
37,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.
55,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions
56,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",,,6
1,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,,,1
2,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC",,,26
3,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.,,,1
4,Jul 13 2022 10:46AM,241757,10,Beach Products Inc,,,1
5,Jul 13 2022 10:43AM,241756,19,"GCH Granules USA, Inc.",,,1
6,Jul 13 2022 10:36AM,241736,10,Bio-PRF,,,1
7,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,,,18
8,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,,,1
9,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",6,,
1,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,1,,
2,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC",26,,
3,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.,1,,
4,Jul 13 2022 10:46AM,241757,10,Beach Products Inc,1,,
5,Jul 13 2022 10:43AM,241756,19,"GCH Granules USA, Inc.",1,,
6,Jul 13 2022 10:36AM,241736,10,Bio-PRF,1,,
7,Jul 13 2022 10:35AM,241753,10,Eywa Pharma Inc.,18,,
8,Jul 13 2022 10:34AM,241755,16,Sartorius Bioprocess Solutions,1,,
9,Jul 13 2022 10:29AM,241752,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",6,,
1,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,1,,
2,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC",26,,
3,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.,1,,
4,Jul 13 2022 10:46AM,241757,10,Beach Products Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",6.0,NaN,NaN
1,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC",26.0,NaN,NaN
3,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
4,Jul 13 2022 10:46AM,241757,10,Beach Products Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",6.0,0.0,0.0
1,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Jul 13 2022 10:53AM,241710,15,"Brookfield Pharmaceuticals, LLC",26.0,0.0,0.0
3,Jul 13 2022 10:47AM,241758,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
4,Jul 13 2022 10:46AM,241757,10,Beach Products Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4834230,132.0,90.0,1.0
12,725235,29.0,1.0,0.0
15,966735,74.0,0.0,0.0
16,1450444,24.0,2.0,0.0
19,1208593,6.0,25.0,4.0
20,1208587,63.0,7.0,0.0
21,725115,3.0,0.0,0.0
22,483493,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4834230,132.0,90.0,1.0
1,12,725235,29.0,1.0,0.0
2,15,966735,74.0,0.0,0.0
3,16,1450444,24.0,2.0,0.0
4,19,1208593,6.0,25.0,4.0
5,20,1208587,63.0,7.0,0.0
6,21,725115,3.0,0.0,0.0
7,22,483493,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,132.0,90.0,1.0
1,12,29.0,1.0,0.0
2,15,74.0,0.0,0.0
3,16,24.0,2.0,0.0
4,19,6.0,25.0,4.0
5,20,63.0,7.0,0.0
6,21,3.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,132.0
1,12,Released,29.0
2,15,Released,74.0
3,16,Released,24.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
Executing,90.0,1.0,0.0,2.0,25.0,7.0,0.0,0.0
Released,132.0,29.0,74.0,24.0,6.0,63.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1,Executing,90.0,1.0,0.0,2.0,25.0,7.0,0.0,0.0
2,Released,132.0,29.0,74.0,24.0,6.0,63.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1,Executing,90.0,1.0,0.0,2.0,25.0,7.0,0.0,0.0
2,Released,132.0,29.0,74.0,24.0,6.0,63.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()